In [1]:
import csv
import math
import random

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, AutoMinorLocator
from qiskit import IBMQ, Aer, execute, ClassicalRegister
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA, CG, ISRES, AQGD, BOBYQA, ADAM, GSLS, NELDER_MEAD, NFT, SLSQP, SPSA, \
    TNC, POWELL
from qiskit.aqua import aqua_globals
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.providers.ibmq import least_busy
from qiskit.utils import QuantumInstance
from qiskit.visualization import plot_histogram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
import numpy as np
import csv
import imageio
import pandas as pd

C:\Users\tobia\anaconda3\envs\QAOA_Experiments\lib\site-packages\qiskit\optimization\__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


# Methods
## What device to use

In [2]:
def use_simulator():
    aqua_globals.random_seed = 69069
    seed = 29
    backend = Aer.get_backend('qasm_simulator')
    return QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed), backend


def use_online_qasm():
    IBMQ.load_account()
    provider = IBMQ.get_provider(hub='ibm-q')
    backend = provider.get_backend('ibmq_qasm_simulator')
    return QuantumInstance(backend, skip_qobj_validation=False), backend


def use_real_device():
    IBMQ.load_account()
    provider = IBMQ.get_provider(hub='ibm-q')
    backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 4 and
                                                             not x.configuration().simulator and x.status().operational == True))
    print("Running on current least busy device: ", backend)
    return QuantumInstance(backend, skip_qobj_validation=False), backend

## Save data from classical optimizer

In [3]:
class Steps:
    def __init__(self):
        self.steps = []

    def next_step(self, eval_count, params, eval_mean, eval_sd):
        self.steps.append([eval_count, params, eval_mean, eval_sd])


def callback(eval_count, params, eval_mean, eval_sd):
    saved_data.next_step(eval_count, params, eval_mean, eval_sd)

## Create QAOA-Instance

In [4]:
def create_qaoa(instance, p, maxiter=1000, params=None):
    optimizer = COBYLA(maxiter=maxiter)
    qaoa_instance = QAOA(optimizer=optimizer, initial_point=params,
                         reps=p, quantum_instance=instance,
                         callback=callback)
    return qaoa_instance

## Create random Problems

In [5]:
def create_savings(nr_of_queries, nr_of_plans):
    savings = {}
    nr_of_plans_each = int(nr_of_plans / nr_of_queries)
    for j in range(nr_of_plans):
        current_query = math.floor(j / nr_of_plans_each)
        first_plan_next_query = (current_query + 1) * nr_of_plans_each
        for i in range(first_plan_next_query, nr_of_plans):
            savings[j, i] = random.randint(-20, 0)

    return savings

## Create operator from problem

In [6]:
def create_problem_matrix_and_dict(problem_tuple):
    nr_of_queries = problem_tuple[0]
    plan_costs = problem_tuple[1]
    savings = problem_tuple[2]

    nr_of_plans = len(plan_costs)
    nr_of_plans_each = nr_of_plans / nr_of_queries

    eps = 1
    w_min = max(plan_costs) + eps
    w_max = w_min
    if savings:
        sum_savings = sum(savings.values())
        w_max = w_min - sum_savings

    linear_terms = []
    quadratic_terms = {}

    for i in range(nr_of_plans):
        for j in range(i, nr_of_plans):
            query_i = math.floor(i / nr_of_plans_each)
            query_j = math.floor(j / nr_of_plans_each)
            plan_1 = 'p' + str(i + 1)
            plan_2 = 'p' + str(j + 1)
            if i == j:
                linear_terms.append(plan_costs[i] - w_min)
            elif query_i == query_j:
                quadratic_terms[plan_1, plan_2] = w_max
            else:
                tuple_forward = (i, j)
                tuple_backward = (j, i)
                if tuple_forward in savings:
                    quadratic_terms[plan_1, plan_2] = savings[tuple_forward]
                elif tuple_backward in savings:
                    quadratic_terms[plan_1, plan_2] = savings[tuple_backward]

    return linear_terms, quadratic_terms


def create_problem_operator(linear_terms, quadratic_terms):
    # create a QUBO
    qubo = QuadraticProgram()
    for i in range(len(linear_terms)):
        qubo.binary_var('p' + str(i + 1))

    qubo.minimize(linear=linear_terms, quadratic=quadratic_terms)

    qubit_op, offset = qubo.to_ising()
    return qubit_op, qubo

## Calc costs from state and problem

In [7]:
def calc_costs(problem, solution_state):
    costs = problem[1]
    savings = problem[2]
    total_costs = 0
    for nr, plan in enumerate(solution_state):
        if plan == 1:
            total_costs += costs[nr]
            for i in range(nr+1, len(solution_state)):
                if (nr, i) in savings and solution_state[i] == 1:
                    total_costs += savings[nr, i]
    return total_costs

## Outputs

In [8]:
def plot_accuracy(accuracy):
    for key in accuracy.keys():
        p = key[0]
        qbs = key[1]
        with open('correct_results.csv', 'a+', newline='') as f:
            writer = csv.writer(f)
            row_content = [p, qbs, accuracy[key]]
            writer.writerow(row_content)
            print(f'Problemsize: {qbs}\nRepetitions: {p}\nAccuracy: {accuracy[key]}')
            

def printProblem(problem):
    print("Costs:")
    print(problem[1])
    print("Savings:")
    print(problem[2])

# Program random initialization

In [9]:
quantum_instance, backend = use_simulator()
# quantum_instance, backend = use_real_device()
# quantum_instance, backend = use_online_qasm()

<ipython-input-2-fbf197805b9e>:2: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.algorithm_globals (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  aqua_globals.random_seed = 69069


In [10]:
problem_sizes_qbs = [4]
nr_of_queries = 1
nr_of_runs_per_problem = 1
max_p = 1
accuracy = {}

In [11]:
for p in range(1, max_p + 1):
    for nr_of_qbs in problem_sizes_qbs:
        for run_nr in range(nr_of_runs_per_problem):     
            # Create Problems -------------------------------
            # (nr_of_queries, cost_vector, savings_dict)
            # (2, [3, 13, 21, 1], {(2, 3): -14})
            problem = (nr_of_queries,  np.random.randint(0, 50, nr_of_qbs), create_savings(nr_of_queries, nr_of_qbs))
            saved_data = Steps()

            # create QUBO-Operator from problem ----------------------
            linear, quadratic = create_problem_matrix_and_dict(problem)
            problem_operator, qubo = create_problem_operator(linear, quadratic)
            # --------------------------------------------------------

            # create QAOA and solve problem --------------------------
            reps = p  # p
            params = None
            qaoa = create_qaoa(quantum_instance, reps, params=params)
            qaoa_mes = MinimumEigenOptimizer(qaoa)
            result = qaoa_mes.solve(qubo)
            solution_state = result.x
            
            opt_params = qaoa.optimal_params
            expectation_value = qaoa.get_optimal_cost()
            q_costs_qaoa = qaoa.get_optimal_cost()
            q_costs_calc = calc_costs(problem, solution_state)
            # --------------------------------------------------------

            # calculate with classical eigensolver -------------------
            npme = NumPyMinimumEigensolver()
            exact = MinimumEigenOptimizer(npme)
            classical_result = exact.solve(qubo)
            classical_solution_state = classical_result.x
            classical_costs = classical_result.fval
            c_costs_calc = calc_costs(problem, classical_solution_state)
            # --------------------------------------------------------
            
            index = (p, nr_of_qbs)
            if q_costs_calc == c_costs_calc:
                print(f"q_sol: {solution_state}, c_sol: {classical_solution_state}")
                if index in accuracy:
                    accuracy[index] += 1/nr_of_runs_per_problem
                else:
                    accuracy[index] = 1/nr_of_runs_per_problem
            else:
                printProblem(problem)
                print(f"c_state: {classical_solution_state}, c_costs: {c_costs_calc}")
                print(f"q_state: {solution_state}, q_costs: {q_costs_calc}")

plot_accuracy(accuracy)

q_sol: [0. 0. 1. 0.], c_sol: [0. 0. 1. 0.]
Problemsize: 4
Repetitions: 1
Accuracy: 1.0


# FOURIER startegie

## Device

In [24]:
quantum_instance, backend = use_simulator()
# quantum_instance, backend = use_real_device()
# quantum_instance, backend = use_online_qasm()

## Parameters

In [13]:
def set_beta_gamma(params):
    u = []
    v = []
    for i, param in enumerate(params):
        if i % 2 == 0:
            u.append(param)
        else:
            v.append(param)
    
    p = len(u)
    
    g_i = 0
    b_i = 0
    params = []
    for i in range(1, p+2):
        for k in range(len(u)):
            g_i += u[k] * math.sin( (k-1/2) * (i-1/2) * math.pi/p )
            b_i += v[k] * math.cos( (k-1/2) * (i-1/2) * math.pi/p )

        params = np.append(params, g_i)
        params = np.append(params, b_i)

    return params


def plot_parameters(params, problem_size):
    beta_history = []
    gamma_history = []
    
    for i, value in enumerate(params):
        if i % 2 == 0:
            history = gamma_history
        else:
            history = beta_history

        history.append(value/math.pi)
            
    fig, ax = plt.subplots()
    beta = beta_history
    x = [i for i in range(len(beta))]
    ax.plot(x, beta, linestyle='-', marker='o', label=f'F')

    ax.legend(loc='center right', fontsize='x-large')    
    ax.set_xlabel('Nr of Fs')
    ax.set_ylabel('parameter value')
    ax.set_title("Beta")
    #plt.savefig(f'beta_{problem_size}.pdf')

    fig, ax = plt.subplots()
    gamma = gamma_history
    x = [i for i in range(len(gamma))]
    ax.plot(x, gamma, linestyle='-', marker='o', label=f'F')
        
    ax.legend(loc='center right', fontsize='x-large')    
    ax.set_xlabel('Nr of Fs')
    ax.set_ylabel('parameter value')
    ax.set_title("Gamma")
    #plt.savefig(f'gamma_{problem_size}.pdf')

In [14]:
def construct_circuit(qaoa_results, operator, nr_of_qb):
    q_circuit = qaoa_results.construct_circuit(qaoa_results.optimal_params, operator)
    q_circuit = q_circuit[0]
    cr = ClassicalRegister(nr_of_qb, 'c')
    q_circuit.add_register(cr)
    q_circuit.measure(range(nr_of_qb), range(nr_of_qb))
    return q_circuit


def run_circuit(qc):
    shots = 1000
    job = execute(qc, backend, shots=shots)
    result = job.result()
    return result.get_counts()

In [27]:
problem_sizes_qbs = [4]
nr_of_queries = 1
nr_of_runs_per_problem = 10
max_p = 1

In [28]:
column_names = ['problem_size']
beta_gamma_columns = [f'beta_gamma_F{i+1}' for i in range(max_p)]
right_solution_columns = [f'right_solution_F{i+1}' for i in range(max_p)]
classical_optimizer_columns = [f'classical_optimizer_steps_F{i+1}' for i in range(max_p)]
column_names += beta_gamma_columns + right_solution_columns + classical_optimizer_columns

In [29]:
solutions_df = pd.DataFrame(columns=column_names)

In [30]:
for nr_of_qbs in problem_sizes_qbs:
    print()
    print("#######")
    print(f"problemsize: {nr_of_qbs}")
    for run in range(nr_of_runs_per_problem):
        print(f"run nr: {run}")
        # Create Problems -------------------------------
        # (nr_of_queries, cost_vector, savings_dict)
        # (2, [3, 13, 21, 1], {(2, 3): -14})
        problem = (nr_of_queries,  np.random.randint(0, 50, nr_of_qbs), create_savings(nr_of_queries, nr_of_qbs))
        saved_data = Steps()

        # create QUBO-Operator from problem ----------------------
        linear, quadratic = create_problem_matrix_and_dict(problem)
        problem_operator, qubo = create_problem_operator(linear, quadratic)
        # --------------------------------------------------------

        # calculate with classical eigensolver -------------------
        npme = NumPyMinimumEigensolver()
        exact = MinimumEigenOptimizer(npme)
        classical_result = exact.solve(qubo)
        classical_solution_state = classical_result.x
        classical_costs = classical_result.fval
        c_costs_calc = calc_costs(problem, classical_solution_state)
        # --------------------------------------------------------

        ############# OPTIMIZATION WITH FOURIER ##################
        params = [np.random.uniform(low=-math.pi, high=math.pi),
                  np.random.uniform(low=-math.pi, high=math.pi)]

        next_row = {column_names[0]:nr_of_qbs}

        for i in range(1, max_p+1):
            print(f"Nr of repetitions: {i}")
            saved_data = Steps()
            # create QAOA and solve problem ----------------------
            reps = i
            qaoa = create_qaoa(quantum_instance, reps, params=params)
            result = qaoa.compute_minimum_eigenvalue(problem_operator)
            solution_state = sample_most_likely(result.eigenstate)
            q_costs = qaoa.get_optimal_cost()
            q_costs_calc = calc_costs(problem, solution_state)

            index = (reps, nr_of_qbs)
            right_result = 0
            if q_costs_calc == c_costs_calc:
                right_result = 1

            next_row[beta_gamma_columns[i-1]] = params
            next_row[right_solution_columns[i-1]] = right_result
            next_row[classical_optimizer_columns[i-1]] = saved_data.steps
            params = set_beta_gamma(qaoa.optimal_params)

        solutions_df = solutions_df.append(next_row, ignore_index=True)


#######
problemsize: 4
run nr: 0
Nr of repetitions: 1
run nr: 1
Nr of repetitions: 1
run nr: 2
Nr of repetitions: 1
run nr: 3
Nr of repetitions: 1
run nr: 4
Nr of repetitions: 1
run nr: 5
Nr of repetitions: 1
run nr: 6
Nr of repetitions: 1
run nr: 7
Nr of repetitions: 1
run nr: 8
Nr of repetitions: 1
run nr: 9
Nr of repetitions: 1


In [31]:
solutions_df.head()

,problem_size,beta_gamma_F1,right_solution_F1,classical_optimizer_steps_F1
0,4,"[-1.3086082038729856, -1.170929766564415]",1,"[[1, [-0.30411320098960626, -1.138002283111646..."
1,4,"[2.4838217396528055, 2.80449306795313]",0,"[[1, [2.473028274421791, 2.7515444866998813], ..."
2,4,"[-0.6197308748229307, -0.9204597180403149]",1,"[[1, [-0.6404541268540429, -0.8395781798795112..."
3,4,"[2.187044894795374, -1.9687040520151204]",1,"[[1, [3.2130550655847436, -1.9275587818898037]..."
4,4,"[-0.46749832191702323, 0.9494489658679504]",0,"[[1, [0.7215080826883091, 2.9333272395061933],..."


In [ ]:
solutions_df.to_csv(r'/solution_dataframe_2.csv', index = False, header = True)